# 1. Привести по 2 примера, когда лучше максимизировать Precision, а когда Recall?

# для ответа на вопрос я привел решение двух задачек

In [0]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from google.colab import drive
import h5py

---------------------------------------------------------------------------------------------------


# recall важнее, например, при диганостике рака - значимость ошибки второго рода крайне высока, поэтому пропуск цели должен быть минимизирован $ FN -> min$ , что подтверждается высоким recall: $$ recall = \frac{tp}{tp + fn} $$

## recall is required

$ recall = \frac{tp}{tp + fn} $ , false negatives are critical

### functions

In [0]:
# measure the prediction

def pred_metrics(model, x_train, x_test, y_train, y_test):
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    cm = confusion_matrix(y_test, y_pred)
    rec = cm[0, 0] / (cm[0, 0] + cm[1, 0])
    prec = cm[0, 0] / (cm[0, 0] + cm[0, 1])
    print(f'recall {rec}, precision {prec}')
    return cm

### data upload

In [3]:
!wget http://ftp.cs.wisc.edu/math-prog/cpo-dataset/machine-learn/cancer/WDBC/WDBC.dat

--2019-11-13 15:47:44--  http://ftp.cs.wisc.edu/math-prog/cpo-dataset/machine-learn/cancer/WDBC/WDBC.dat
Resolving ftp.cs.wisc.edu (ftp.cs.wisc.edu)... 128.105.2.31
Connecting to ftp.cs.wisc.edu (ftp.cs.wisc.edu)|128.105.2.31|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 124103 (121K) [text/plain]
Saving to: ‘WDBC.dat.1’

WDBC.dat.1          100%[===================>] 121.19K   453KB/s    in 0.3s    

2019-11-13 15:47:45 (453 KB/s) - ‘WDBC.dat.1’ saved [124103/124103]



In [4]:
!ls

bank.csv  sample_data  WDBC.dat  WDBC.dat.1


In [5]:
df = pd.read_csv('WDBC.dat', header=None)
print(df.shape)
df.head(3)

(569, 32)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,842302,M,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.0,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.5,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758


### preprocessing

In [6]:
df.rename(columns={0 : 'id', 1 : 'diagnosis'}, inplace=True)
df.set_index('id', inplace=True)
df.head()

,diagnosis,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [7]:
# checking nans

df.isnull().sum()
df.isna().sum()

diagnosis    0
2            0
3            0
4            0
5            0
6            0
7            0
8            0
9            0
10           0
11           0
12           0
13           0
14           0
15           0
16           0
17           0
18           0
19           0
20           0
21           0
22           0
23           0
24           0
25           0
26           0
27           0
28           0
29           0
30           0
31           0
dtype: int64

In [8]:
# encoding categorical values

encoder = LabelEncoder()
df.diagnosis = encoder.fit_transform(df.diagnosis)
print(df.diagnosis.unique())

[1 0]


In [0]:
# segregating features and targets

x = df.loc[:, df.columns != 'diagnosis'].values
y = df.diagnosis.values

In [0]:
# scaling features

scaler = StandardScaler()
x = scaler.fit_transform(x)

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

### fit & predict

In [12]:
# log regression

model = LogisticRegression(random_state=42)
pred_metrics(model, x_train, x_test, y_train, y_test)

recall 0.9886363636363636, precision 0.9775280898876404


array([[87,  2],
       [ 1, 53]])

In [13]:
# k neighbors

model = KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2)
pred_metrics(model, x_train, x_test, y_train, y_test)

recall 0.9662921348314607, precision 0.9662921348314607


array([[86,  3],
       [ 3, 51]])

In [14]:
# linear svc

model = SVC(kernel='linear', random_state=42)
pred_metrics(model, x_train, x_test, y_train, y_test)

recall 0.9886363636363636, precision 0.9775280898876404


array([[87,  2],
       [ 1, 53]])

In [15]:
# radial basis svc

model = SVC(kernel='rbf', random_state=42)
pred_metrics(model, x_train, x_test, y_train, y_test)

recall 0.9775280898876404, precision 0.9775280898876404


array([[87,  2],
       [ 2, 52]])

In [16]:
# gaussian naive bayes

model = GaussianNB()
pred_metrics(model, x_train, x_test, y_train, y_test)

recall 0.9659090909090909, precision 0.9550561797752809


array([[85,  4],
       [ 3, 51]])

In [17]:
# decision trees

model = DecisionTreeClassifier(criterion='entropy', random_state=42)
pred_metrics(model, x_train, x_test, y_train, y_test)

recall 0.9560439560439561, precision 0.9775280898876404


array([[87,  2],
       [ 4, 50]])

In [18]:
# random forest

model = RandomForestClassifier(n_estimators=10, criterion='entropy', random_state=42)
pred_metrics(model, x_train, x_test, y_train, y_test)

recall 0.9662921348314607, precision 0.9662921348314607


array([[86,  3],
       [ 3, 51]])

for this type of task the highest recall is required thus either the log regression or the SVC with the linear kernel will be preferrable

возьмем лог регрессию и метод опорных векторов , т.к. они максимизируют recall

---------------------------------------------------------------------------------------------------


# precision важнее, например, для рекомендательных систем по продуктам при относительно высоких затратах на из предложение (временных, финансовых и т.д.) - значимость ошибки второго рода не так критична, а ошибка первого рода будет стоить определенных ресурсов $ FP -> min$ , что подтверждается высоким recall: $$ precision = \frac{tp}{tp + fp} $$

## precision is required

$ precision = \frac{tp}{tp + fp} $ , false negatives are not critical

### data upload

In [19]:
!wget 'https://raw.githubusercontent.com/DataScience86/Data-Science-Beginners/master/Recommender%20Systems%20-%20Python/data/bank.csvhttps://raw.githubusercontent.com/DataScience86/Data-Science-Beginners/master/Recommender%20Systems%20-%20Python/data/bank.csv'

--2019-11-13 15:47:49--  https://raw.githubusercontent.com/DataScience86/Data-Science-Beginners/master/Recommender%20Systems%20-%20Python/data/bank.csvhttps://raw.githubusercontent.com/DataScience86/Data-Science-Beginners/master/Recommender%20Systems%20-%20Python/data/bank.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5515089 (5.3M) [text/plain]
Saving to: ‘bank.csv.1’

bank.csv.1          100%[===================>]   5.26M  --.-KB/s    in 0.07s   

2019-11-13 15:47:49 (77.5 MB/s) - ‘bank.csv.1’ saved [5515089/5515089]



In [20]:
df = pd.read_csv('bank.csv')
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,y_binary,housing_loan,credit_in_default,personal_loans,prev_failed_to_subscribe,prev_subscribed,job_management,job_tech,job_entrepreneur,job_bluecollar,job_unknown,job_retired,job_services,job_self_employed,job_unemployed,job_maid,job_student,married,single,divorced
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1


### preprocessing

In [0]:
df.drop(columns=['y'], inplace=True)

In [22]:
encoder = LabelEncoder()
df.y = encoder.fit_transform(df['y_binary                    '])
print(df['y_binary                    '].unique())

[0 1]


In [0]:
df = pd.get_dummies(df)

In [24]:
df.columns

Index(['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous',
       'y_binary                    ', 'housing_loan                ',
       'credit_in_default', 'personal_loans', 'prev_failed_to_subscribe    ',
       'prev_subscribed             ', 'job_management              ',
       'job_tech                    ', 'job_entrepreneur            ',
       'job_bluecollar              ', 'job_unknown                 ',
       'job_retired                 ', 'job_services                ',
       'job_self_employed           ', 'job_unemployed              ',
       'job_maid                    ', 'job_student                 ',
       'married                     ', 'single                      ',
       'divorced                    ', 'job_admin.', 'job_blue-collar',
       'job_entrepreneur', 'job_housemaid', 'job_management', 'job_retired',
       'job_self-employed', 'job_services', 'job_student', 'job_technician',
       'job_unemployed', 'job_unknown', 'marital_di

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 71 columns):
age                             45211 non-null int64
balance                         45211 non-null int64
day                             45211 non-null int64
duration                        45211 non-null int64
campaign                        45211 non-null int64
pdays                           45211 non-null int64
previous                        45211 non-null int64
y_binary                        45211 non-null int64
housing_loan                    45211 non-null int64
credit_in_default               45211 non-null int64
personal_loans                  45211 non-null int64
prev_failed_to_subscribe        45211 non-null int64
prev_subscribed                 45211 non-null int64
job_management                  45211 non-null int64
job_tech                        45211 non-null int64
job_entrepreneur                45211 non-null int64
job_bluecollar                  45211 non-n

In [26]:
df.shape

(45211, 71)

In [0]:
x = df.loc[:, df.columns != 'y_binary                    '].values
y = df['y_binary                    '].values

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

### fit & predict

In [29]:
# log regression

model = LogisticRegression(random_state=42)
pred_metrics(model, x_train, x_test, y_train, y_test)

recall 0.9170927397152521, precision 0.9743439745946849


array([[11659,   307],
       [ 1054,   544]])

In [30]:
# k neighbors

model = KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2)
pred_metrics(model, x_train, x_test, y_train, y_test)

recall 0.9072482056944554, precision 0.9613070366037105


array([[11503,   463],
       [ 1176,   422]])

In [31]:
# gaussian naive bayes

model = GaussianNB()
pred_metrics(model, x_train, x_test, y_train, y_test)

recall 0.9333861316437874, precision 0.8864282132709344


array([[10607,  1359],
       [  757,   841]])

In [32]:
# decision trees

model = DecisionTreeClassifier(criterion='entropy', random_state=42)
pred_metrics(model, x_train, x_test, y_train, y_test)

recall 0.9283333333333333, precision 0.9309710847400969


array([[11140,   826],
       [  860,   738]])

In [33]:
# random forest

model = RandomForestClassifier(n_estimators=10, criterion='entropy', random_state=42)
pred_metrics(model, x_train, x_test, y_train, y_test)

recall 0.9163921568627451, precision 0.9764332274778539


array([[11684,   282],
       [ 1066,   532]])

for this type of task the highest precision is required thus the random forest will be preferrable

берем случайный лес, т.к. он максимизирует precision

---------------------------------------------------------------------------------------------------


# при этом выбор архитектуры модели должен проводиться исзодя из метрики f score , для которой значчимость recall / precision регулируется через коэф

# 2. 2. Почему мы используем F-меру, почему, например, нельзя просто взять среднее от Precision и Recall?

предполагаю, простой ответ - в метрике f score мы хотим балансировать между precision и recall через коэффициент $\beta$

сложный: согласно теории измерений обобщающий показатель должен обладать следующими свойствами:
- связанность (2 пары могут быть упорядочены) и транизитивность
- независимость (2 сооставляющие вносят свой вклад независимо друг от друга)
- соблюдения условия томсона (если при константе одной из составляющей мы определяем разность эффективности двух значений данной метрики, то разность не может быть изменена через изменение данного константного значения)
- каждый компонент является необходимым
- соблюдение принципа архимеда для каждого компонента (Если имеются две величины, a и b, и a меньше b, то, взяв a слагаемым достаточное количество раз, можно превзойти b, интервалы по каждой компоненте сравнимы)